In [186]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from pdpbox.pdp import pdp_isolate, pdp_plot
import shap

In [247]:
# lets you see all columns

pd.set_option('display.max_columns', None)

In [187]:
# imports the csv from my github as a dataframe

df0 = pd.read_csv('https://raw.githubusercontent.com/SeanAntosiak/LS-DS6-Unit-2-Project/master/LoLgames.csv')

In [248]:
# looks at dataframe

df0.head()

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,t1_champ1_sum1,t1_champ1_sum2,t1_champ2id,t1_champ2_sum1,t1_champ2_sum2,t1_champ3id,t1_champ3_sum1,t1_champ3_sum2,t1_champ4id,t1_champ4_sum1,t1_champ4_sum2,t1_champ5id,t1_champ5_sum1,t1_champ5_sum2,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t1_ban1,t1_ban2,t1_ban3,t1_ban4,t1_ban5,t2_champ1id,t2_champ1_sum1,t2_champ1_sum2,t2_champ2id,t2_champ2_sum1,t2_champ2_sum2,t2_champ3id,t2_champ3_sum1,t2_champ3_sum2,t2_champ4id,t2_champ4_sum1,t2_champ4_sum2,t2_champ5id,t2_champ5_sum1,t2_champ5_sum2,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,2,8,12,4,432,3,4,96,4,7,11,11,6,112,4,14,11,1,2,3,0,92,40,69,119,141,104,11,4,498,4,7,122,6,4,238,14,4,412,4,3,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,1,119,7,4,39,12,4,76,4,3,10,4,14,35,4,11,10,4,0,2,1,51,122,17,498,19,54,4,12,25,4,14,120,11,4,157,4,14,92,4,7,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,0,18,4,7,141,11,4,267,3,4,68,4,12,38,12,4,8,1,1,1,0,117,40,29,16,53,69,4,7,412,14,4,126,4,12,24,4,11,22,7,4,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,0,57,4,12,63,4,14,29,4,7,61,4,1,36,11,4,9,2,1,2,0,238,67,516,114,31,90,14,4,19,11,4,412,4,3,92,4,14,22,4,7,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,0,19,4,12,29,11,4,40,4,3,119,4,7,134,7,4,9,2,1,3,0,90,64,412,25,31,37,3,4,59,4,12,141,11,4,38,4,12,51,4,7,3,0,0,1,0,86,11,201,122,18


In [189]:
# looks at columns

df0.columns

Index(['gameId', 'creationTime', 'gameDuration', 'seasonId', 'winner',
       'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron',
       'firstDragon', 'firstRiftHerald', 't1_champ1id', 't1_champ1_sum1',
       't1_champ1_sum2', 't1_champ2id', 't1_champ2_sum1', 't1_champ2_sum2',
       't1_champ3id', 't1_champ3_sum1', 't1_champ3_sum2', 't1_champ4id',
       't1_champ4_sum1', 't1_champ4_sum2', 't1_champ5id', 't1_champ5_sum1',
       't1_champ5_sum2', 't1_towerKills', 't1_inhibitorKills', 't1_baronKills',
       't1_dragonKills', 't1_riftHeraldKills', 't1_ban1', 't1_ban2', 't1_ban3',
       't1_ban4', 't1_ban5', 't2_champ1id', 't2_champ1_sum1', 't2_champ1_sum2',
       't2_champ2id', 't2_champ2_sum1', 't2_champ2_sum2', 't2_champ3id',
       't2_champ3_sum1', 't2_champ3_sum2', 't2_champ4id', 't2_champ4_sum1',
       't2_champ4_sum2', 't2_champ5id', 't2_champ5_sum1', 't2_champ5_sum2',
       't2_towerKills', 't2_inhibitorKills', 't2_baronKills', 't2_dragonKills',
       't2_riftHer

In [190]:
# checks shape of dataframe

df0.shape

(51490, 61)

In [191]:
# looking at the number of unique values in the first 5 columns

df0.nunique().head()

# gameId has less values than the shape of the dataframe so there must be duplicates which I will drop
# all games are season 9, so I will drop this column later
# creation time  is almost as unique as games, so I will also drop this column

gameId          51053
creationTime    51052
gameDuration     2590
seasonId            1
winner              2
dtype: int64

In [192]:
# checks the shape of the dataframe after droping duplicates

df0.drop_duplicates().shape

# since the rows of this DF matches the number of unique gameId values, I will use it as the new dataframe
# and then remove the game Id column since each game has its own unique id

(51053, 61)

In [275]:
# creates a copy of the dataframe and then drops rows/columns before mentioned columns

df = df0.copy()

df.drop_duplicates(inplace=True)
df.drop(columns=['gameId', 'creationTime', 'seasonId'], inplace=True)

In [276]:
# creates a dictionary to name teams then applies it to a few different columns
# then checks to make sure it worked

teamColor = {1: 'blue', 2: 'red', 0: 'neither'}
df.replace({'winner': teamColor, 
            'firstBlood': teamColor, 
            'firstTower': teamColor, 
            'firstInhibitor': teamColor, 
            'firstBaron': teamColor, 
            'firstDragon': teamColor, 
            'firstRiftHerald': teamColor}, inplace = True)
df.head()

,gameDuration,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,t1_champ1_sum1,t1_champ1_sum2,t1_champ2id,t1_champ2_sum1,t1_champ2_sum2,t1_champ3id,t1_champ3_sum1,t1_champ3_sum2,t1_champ4id,t1_champ4_sum1,t1_champ4_sum2,t1_champ5id,t1_champ5_sum1,t1_champ5_sum2,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t1_ban1,t1_ban2,t1_ban3,t1_ban4,t1_ban5,t2_champ1id,t2_champ1_sum1,t2_champ1_sum2,t2_champ2id,t2_champ2_sum1,t2_champ2_sum2,t2_champ3id,t2_champ3_sum1,t2_champ3_sum2,t2_champ4id,t2_champ4_sum1,t2_champ4_sum2,t2_champ5id,t2_champ5_sum1,t2_champ5_sum2,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,1949,blue,red,blue,blue,blue,blue,red,8,12,4,432,3,4,96,4,7,11,11,6,112,4,14,11,1,2,3,0,92,40,69,119,141,104,11,4,498,4,7,122,6,4,238,14,4,412,4,3,5,0,0,1,1,114,67,43,16,51
1,1851,blue,blue,blue,blue,neither,blue,blue,119,7,4,39,12,4,76,4,3,10,4,14,35,4,11,10,4,0,2,1,51,122,17,498,19,54,4,12,25,4,14,120,11,4,157,4,14,92,4,7,2,0,0,0,0,11,67,238,51,420
2,1493,blue,red,blue,blue,blue,red,neither,18,4,7,141,11,4,267,3,4,68,4,12,38,12,4,8,1,1,1,0,117,40,29,16,53,69,4,7,412,14,4,126,4,12,24,4,11,22,7,4,2,0,0,1,0,157,238,121,57,28
3,1758,blue,blue,blue,blue,blue,blue,neither,57,4,12,63,4,14,29,4,7,61,4,1,36,11,4,9,2,1,2,0,238,67,516,114,31,90,14,4,19,11,4,412,4,3,92,4,14,22,4,7,0,0,0,0,0,164,18,141,40,51
4,2094,blue,red,blue,blue,blue,blue,neither,19,4,12,29,11,4,40,4,3,119,4,7,134,7,4,9,2,1,3,0,90,64,412,25,31,37,3,4,59,4,12,141,11,4,38,4,12,51,4,7,3,0,0,1,0,86,11,201,122,18


In [277]:
# changing the names of the columns with t1/t2 to have blue/red instead

df.columns=['gameDuration', 'winner', 'firstBlood', 'firstTower', 'firstInhibitor',
       'firstBaron', 'firstDragon', 'firstRiftHerald', 'blue_champ1id',
       'blue_champ1_sum1', 'blue_champ1_sum2', 'blue_champ2id', 'blue_champ2_sum1',
       'blue_champ2_sum2', 'blue_champ3id', 'blue_champ3_sum1', 'blue_champ3_sum2',
       'blue_champ4id', 'blue_champ4_sum1', 'blue_champ4_sum2', 'blue_champ5id',
       'blue_champ5_sum1', 'blue_champ5_sum2', 'blue_towerKills',
       'blue_inhibitorKills', 'blue_baronKills', 'blue_dragonKills',
       'blue_riftHeraldKills', 'blue_ban1', 'blue_ban2', 'blue_ban3', 'blue_ban4',
       'blue_ban5', 'red_champ1id', 'red_champ1_sum1', 'red_champ1_sum2',
       'red_champ2id', 'red_champ2_sum1', 'red_champ2_sum2', 'red_champ3id',
       'red_champ3_sum1', 'red_champ3_sum2', 'red_champ4id', 'red_champ4_sum1',
       'red_champ4_sum2', 'red_champ5id', 'red_champ5_sum1', 'red_champ5_sum2',
       'red_towerKills', 'red_inhibitorKills', 'red_baronKills', 'red_dragonKills',
       'red_riftHeraldKills', 'red_ban1', 'red_ban2', 'red_ban3', 'red_ban4',
       'red_ban5']

In [278]:
df.columns

Index(['gameDuration', 'winner', 'firstBlood', 'firstTower', 'firstInhibitor',
       'firstBaron', 'firstDragon', 'firstRiftHerald', 'blue_champ1id',
       'blue_champ1_sum1', 'blue_champ1_sum2', 'blue_champ2id',
       'blue_champ2_sum1', 'blue_champ2_sum2', 'blue_champ3id',
       'blue_champ3_sum1', 'blue_champ3_sum2', 'blue_champ4id',
       'blue_champ4_sum1', 'blue_champ4_sum2', 'blue_champ5id',
       'blue_champ5_sum1', 'blue_champ5_sum2', 'blue_towerKills',
       'blue_inhibitorKills', 'blue_baronKills', 'blue_dragonKills',
       'blue_riftHeraldKills', 'blue_ban1', 'blue_ban2', 'blue_ban3',
       'blue_ban4', 'blue_ban5', 'red_champ1id', 'red_champ1_sum1',
       'red_champ1_sum2', 'red_champ2id', 'red_champ2_sum1', 'red_champ2_sum2',
       'red_champ3id', 'red_champ3_sum1', 'red_champ3_sum2', 'red_champ4id',
       'red_champ4_sum1', 'red_champ4_sum2', 'red_champ5id', 'red_champ5_sum1',
       'red_champ5_sum2', 'red_towerKills', 'red_inhibitorKills',
       'red_baron

In [279]:
# importing champion/spell id values/names from json in my github

r1 = requests.get('https://raw.githubusercontent.com/SeanAntosiak/LS-DS6-Unit-2-Project/master/IDs/champion_info.json')
j1 = json.loads(r1.content)

r2 = requests.get('https://raw.githubusercontent.com/SeanAntosiak/LS-DS6-Unit-2-Project/master/IDs/summoner_spell_info.json')
j2 = json.loads(r2.content)

In [280]:
# checking the length of bluechamp1 selection

len(df['blue_champ1id'].value_counts())

# all champions were picked in this column so I will us it to
# make a list of all champ ids

138

In [281]:
# checking the length of the first champions summoner spells

len(df['blue_champ1_sum1'].value_counts())

# all spells were picked in this column so I will us it to
# make a list of all spell ids

9

In [282]:
# creating a list of all unique values for champions/spells

champIdList = list(df['blue_champ1id'].unique())
spellIdList = list(df['blue_champ1_sum1'].unique())

In [339]:
# creates a champion id dictonary

champID = {}

for ID in champIdList:
    champID[ID]=j1['data'][f'{ID}']['name'];
    
    
# when no champion is banned there is a value of -1
# manualy adding this to account for when someone neglects to ban
champID[-1]='None'

In [340]:
# creates a spell id dictonary

spellID = {}

for ID in spellIdList:
    spellID[ID]=j2['data'][f'{ID}']['name'];

In [341]:
# creates colums to have dictonaries applied to

ChampCols = ['blue_champ1id','blue_champ2id','blue_champ3id','blue_champ4id','blue_champ5id',
             'blue_ban1', 'blue_ban2', 'blue_ban3','blue_ban4', 'blue_ban5',
             'red_champ1id', 'red_champ2id','red_champ3id','red_champ4id','red_champ5id',
             'red_ban1', 'red_ban2', 'red_ban3','red_ban4', 'red_ban5']

SpellCols = ['blue_champ1_sum1', 'blue_champ1_sum2', 'blue_champ2_sum1', 'blue_champ2_sum2', 
             'blue_champ3_sum1', 'blue_champ3_sum2', 'blue_champ4_sum1', 'blue_champ4_sum2', 
             'blue_champ5_sum1', 'blue_champ5_sum2', 'red_champ1_sum1', 'red_champ1_sum2', 
             'red_champ2_sum1', 'red_champ2_sum2', 'red_champ3_sum1', 'red_champ3_sum2', 
             'red_champ4_sum1', 'red_champ4_sum2', 'red_champ5_sum1', 'red_champ5_sum2']

In [342]:
# aplplies the dictonary to the columns

for col in ChampCols:
    df.replace({col: champID}, inplace=True);
    
for col in SpellCols:
    df.replace({col: spellID}, inplace=True);

In [343]:
# checks if it worked

df.head()

,gameDuration,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,blue_champ1id,blue_champ1_sum1,blue_champ1_sum2,blue_champ2id,blue_champ2_sum1,blue_champ2_sum2,blue_champ3id,blue_champ3_sum1,blue_champ3_sum2,blue_champ4id,blue_champ4_sum1,blue_champ4_sum2,blue_champ5id,blue_champ5_sum1,blue_champ5_sum2,blue_towerKills,blue_inhibitorKills,blue_baronKills,blue_dragonKills,blue_riftHeraldKills,blue_ban1,blue_ban2,blue_ban3,blue_ban4,blue_ban5,red_champ1id,red_champ1_sum1,red_champ1_sum2,red_champ2id,red_champ2_sum1,red_champ2_sum2,red_champ3id,red_champ3_sum1,red_champ3_sum2,red_champ4id,red_champ4_sum1,red_champ4_sum2,red_champ5id,red_champ5_sum1,red_champ5_sum2,red_towerKills,red_inhibitorKills,red_baronKills,red_dragonKills,red_riftHeraldKills,red_ban1,red_ban2,red_ban3,red_ban4,red_ban5
0,1949,blue,red,blue,blue,blue,blue,red,Vladimir,Teleport,Flash,Bard,Exhaust,Flash,Kog'Maw,Flash,Heal,Master Yi,Smite,Ghost,Viktor,Flash,Ignite,11,1,2,3,0,Riven,Janna,Cassiopeia,Draven,Kayn,Graves,Smite,Flash,Xayah,Flash,Heal,Darius,Ghost,Flash,Zed,Ignite,Flash,Thresh,Flash,Exhaust,5,0,0,1,1,Fiora,Vayne,Karma,Soraka,Caitlyn
1,1851,blue,blue,blue,blue,neither,blue,blue,Draven,Heal,Flash,Irelia,Teleport,Flash,Nidalee,Flash,Exhaust,Kayle,Flash,Ignite,Shaco,Flash,Smite,10,4,0,2,1,Caitlyn,Darius,Teemo,Xayah,Warwick,Malphite,Flash,Teleport,Morgana,Flash,Ignite,Hecarim,Smite,Flash,Yasuo,Flash,Ignite,Riven,Flash,Heal,2,0,0,0,0,Master Yi,Vayne,Zed,Caitlyn,Illaoi
2,1493,blue,red,blue,blue,blue,red,neither,Tristana,Flash,Heal,Kayn,Smite,Flash,Nami,Exhaust,Flash,Rumble,Flash,Teleport,Kassadin,Teleport,Flash,8,1,1,1,0,Lulu,Janna,Twitch,Soraka,Blitzcrank,Cassiopeia,Flash,Heal,Thresh,Ignite,Flash,Jayce,Flash,Teleport,Jax,Flash,Smite,Ashe,Heal,Flash,2,0,0,1,0,Yasuo,Zed,Kha'Zix,Maokai,Evelynn
3,1758,blue,blue,blue,blue,blue,blue,neither,Maokai,Flash,Teleport,Brand,Flash,Ignite,Twitch,Flash,Heal,Orianna,Flash,Cleanse,Dr. Mundo,Smite,Flash,9,2,1,2,0,Zed,Vayne,Ornn,Fiora,Cho'Gath,Malzahar,Ignite,Flash,Warwick,Smite,Flash,Thresh,Flash,Exhaust,Riven,Flash,Ignite,Ashe,Flash,Heal,0,0,0,0,0,Camille,Tristana,Kayn,Janna,Caitlyn
4,2094,blue,red,blue,blue,blue,blue,neither,Warwick,Flash,Teleport,Twitch,Smite,Flash,Janna,Flash,Exhaust,Draven,Flash,Heal,Syndra,Heal,Flash,9,2,1,3,0,Malzahar,Lee Sin,Thresh,Morgana,Cho'Gath,Sona,Exhaust,Flash,Jarvan IV,Flash,Teleport,Kayn,Smite,Flash,Kassadin,Flash,Teleport,Caitlyn,Flash,Heal,3,0,0,1,0,Garen,Master Yi,Braum,Darius,Tristana


In [349]:
def reduceFeats(df):

    # creates lists for use in for loops
    players = [1,2,3,4,5]
    teams = ['blue', 'red']

    
    
    # creates a list for each teams Champions
    for team in teams:
        df[f'{team}Champs'] = df[f'{team}_champ1id'] + ',' + df[f'{team}_champ2id']+ ',' + df[f'{team}_champ3id']+ ',' + df[f'{team}_champ4id']+ ',' + df[f'{team}_champ5id']
        df[f'{team}Champs'] = df[f'{team}Champs'].str.split(',')
        
        
        
    # creates a list for each teams bans
    for team in teams:
        df[f'{team}Bans'] = df[f'{team}_ban1'] + ',' + df[f'{team}_ban2']+ ',' + df[f'{team}_ban3']+ ',' + df[f'{team}_ban4']+ ',' + df[f'{team}_ban5']
        df[f'{team}Bans'] = df[f'{team}Bans'].str.split(',')
    
    
    
    # creates a list for each players spells then removes original columns
    for team in teams:
        for player in players:
            df[f'{team}_champ{player}_sums']=df[f'{team}_champ{player}_sum1']+ ',' + df[f'{team}_champ{player}_sum2']
            df[f'{team}_champ{player}_sums']=df[f'{team}_champ{player}_sums'].str.split(',');
            
    # removes original ban/champion columns
    for team in teams:
        for player in players:
            df.drop(columns=[f'{team}_champ{player}_sum1', f'{team}_champ{player}_sum2',
                             f'{team}_champ{player}id', f'{team}_ban{player}'], inplace=True)
    
            
    return(df);

gameDuration             int64
winner                  object
firstBlood              object
firstTower              object
firstInhibitor          object
firstBaron              object
firstDragon             object
firstRiftHerald         object
blue_champ1id           object
blue_champ1_sum1        object
blue_champ1_sum2        object
blue_champ2id           object
blue_champ2_sum1        object
blue_champ2_sum2        object
blue_champ3id           object
blue_champ3_sum1        object
blue_champ3_sum2        object
blue_champ4id           object
blue_champ4_sum1        object
blue_champ4_sum2        object
blue_champ5id           object
blue_champ5_sum1        object
blue_champ5_sum2        object
blue_towerKills          int64
blue_inhibitorKills      int64
blue_baronKills          int64
blue_dragonKills         int64
blue_riftHeraldKills     int64
blue_ban1               object
blue_ban2               object
blue_ban3               object
blue_ban4               object
blue_ban

In [207]:
# checks for a majority class Baseline

df['winner'].value_counts(normalize=True)

# blue team apears to win slightly more games than red team
# baseline model is ~51% accuracy always guessing blue team will win for game at unkown rank

blue    0.506474
red     0.493526
Name: winner, dtype: float64

In [46]:
# creates a function that takes in a dataframe and a target as inputs
# the function will remove the target column and treat all other columns as features
# the function then performs a train test split, and then a train validation split and fits a random forest model
# next it scores it and returns the score, and importance values
# this function will be used to quikcly check accuracys by switching certain columns/features in the input dataframe

def SplitFitForest(df, targ):

    X0 = df.drop(columns=[targ])
    y0 = df[str(targ)]

    X0train, X0test, y0train, y0test = tts(X0,y0, train_size=0.90, test_size=0.10, random_state=8)

    X1train, X1val, y1train, y0val = tts(X0train,y0train, train_size=0.90, test_size=0.10, random_state=8)

    pipe = make_pipeline(ce.OneHotEncoder(use_cat_names=True),
                         RandomForestClassifier(max_depth=20, n_estimators=1000, n_jobs=-1))
                     
    pipe.fit(X1train,y1train)
    
    score = pipe.score(X1val, y0val)    
    
    mod = pipe.named_steps['randomforestclassifier']
    importanceVals = mod.feature_importances_
    importanceCols = pipe.named_steps['onehotencoder'].fit_transform(X1train).columns
    importanceDF = pd.DataFrame(importanceVals, importanceCols).reset_index()
    importanceDF.columns = ['feature', 'importance']
    
    
    return(score, importanceDF);

In [51]:
# runs the function and saves results in new variables

dftry = df[['gameDuration','firstBlood','firstTower','firstInhibitor','firstBaron','firstDragon','firstRiftHerald',
        't1_towerKills', 't1_inhibitorKills', 't1_baronKills','t1_dragonKills', 't1_riftHeraldKills', 'winner' ]]

score0, importanceDF0 = SplitFitForest(dftry, 'winner')

In [55]:
# checks if function worked 

print('accuracy score:', score0)
print('')
print('importance values:')
importanceDF0.T

accuracy score: 0.9536452665941241

importance values:


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
feature,gameDuration,firstBlood_red,firstBlood_neither,firstBlood_blue,firstTower_red,firstTower_blue,firstTower_neither,firstInhibitor_red,firstInhibitor_blue,firstInhibitor_neither,...,firstDragon_blue,firstDragon_neither,firstRiftHerald_neither,firstRiftHerald_red,firstRiftHerald_blue,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills
importance,0.0986287,0.00500113,0.000431424,0.00439333,0.0280407,0.0189444,0.00201372,0.146518,0.083239,0.0239973,...,0.00813173,0.00242128,0.00311885,0.00457909,0.00274505,0.238605,0.183247,0.0235444,0.0487971,0.00245003


412    1273
18     1251
67     1052
141    1004
40      918
64      914
29      872
157     794
236     781
222     773
202     729
99      714
238     703
24      696
61      681
498     675
53      651
11      649
79      643
92      638
59      630
103     625
51      615
497     604
117     598
25      589
16      586
21      563
19      561
81      537
       ... 
56      180
74      180
223     178
76      173
101     169
2       165
50      164
28      160
83      158
104     154
420     152
36      149
133     148
68      147
26      143
27      143
20      132
106     132
82      128
268     125
203     122
421     115
127     115
13      114
77      106
427     105
30       98
136      86
266      68
72       49
Name: t1_champ5id, Length: 138, dtype: int64

In [50]:
# takes the steps out of the function used previously
# and creates a new DF with all features excluding champion/spell choices
    
X0 = df[['gameDuration','firstBlood','firstTower','firstInhibitor','firstBaron','firstDragon','firstRiftHerald',
        't1_towerKills', 't1_inhibitorKills', 't1_baronKills','t1_dragonKills', 't1_riftHeraldKills' ]]

y0 = df[['winner']]

X0train, X0test, y0train, y0test = tts(X0,y0, train_size=0.90, test_size=0.10, random_state=8)

X1train, X1val, y1train, y0val = tts(X0train,y0train, train_size=0.90, test_size=0.10, random_state=8)

pipe = make_pipeline(ce.OneHotEncoder(use_cat_names=True),
                         RandomForestClassifier(max_depth=20, n_estimators=1000, n_jobs=-1))                   
pipe.fit(X1train,y1train)
    
score = pipe.score(X1val, y0val)    
    
mod = pipe.named_steps['randomforestclassifier']
importanceVals = mod.feature_importances_
importanceCols = pipe.named_steps['onehotencoder'].fit_transform(X1train).columns
importanceDF = pd.DataFrame(importanceVals, importanceCols).reset_index()
importanceDF.columns = ['feature', 'importance']

In [45]:
importanceDF

,feature,importance
0,gameDuration,0.099403
1,firstBlood_red,0.005087
2,firstBlood_neither,0.000489
3,firstBlood_blue,0.004581
4,firstTower_red,0.026955
5,firstTower_blue,0.019849
6,firstTower_neither,0.001950
7,firstInhibitor_red,0.130324
8,firstInhibitor_blue,0.089041
9,firstInhibitor_neither,0.025944


In [ ]:
explainer = shap.TreeExplainer(mod)
row_processed = processor.transform(row)
shap_values = explainer.shap_values(row_processed)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values, 
    features=row)